PREDICTION SCRIPT

The best algorithm for prediction has been found to be RandomTreeRegressor. We are going to fit it to the dataset and then just predict the values and save them.

Importing dataset and test set

In [4]:
from holoviews.streams import Lasso
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

In [11]:
df = pd.read_csv('dataset_fase2.csv')
test_df = pd.read_csv('dataset_test_fase2.csv')
target = df.iloc[:, 41]
target_test = test_df.iloc[:, 41]
#drop all teh columns except 'Turbidez fin cultivo', 'media_PV', 'Centrifugación 1 turbidez','Volumen de inóculo utilizado', 'Turbidez final culttivo_inoculo','Centrifugación 2 turbidez', 'media_temp_bioreactor','Turbidez inicio cultivo_cultivo_final', 'media_ph_biorreactor',       'Viabilidad final cultivo']

df_dropped = df.loc[:, ['Turbidez fin cultivo', 'media_PV', 'Centrifugación 1 turbidez','Volumen de inóculo utilizado', 'Turbidez final culttivo_inoculo','Centrifugación 2 turbidez', 'media_temp_bioreactor','Turbidez inicio cultivo_cultivo_final', 'media_ph_biorreactor',       'Viabilidad final cultivo']]

df_test_dropped = test_df.loc[:, ['Turbidez fin cultivo', 'media_PV', 'Centrifugación 1 turbidez','Volumen de inóculo utilizado', 'Turbidez final culttivo_inoculo','Centrifugación 2 turbidez', 'media_temp_bioreactor','Turbidez inicio cultivo_cultivo_final', 'media_ph_biorreactor',       'Viabilidad final cultivo']]


In [6]:
"""
columns_to_drop = df.loc[:,].columns
df_dropped = df.drop(columns=columns_to_drop)
df_test_dropped = test_df.drop(columns=columns_to_drop)
"""

In [9]:
max(target)

2395.36

In [18]:
def clean_and_convert(column):
    column = column.str.replace(',', '.', regex=False)
    column = column.str.replace('+', '', regex=False)
    return pd.to_numeric(column, errors='coerce')

def data_nan(data):
    data.replace('N.A', np.nan, inplace=True)
    #data['línea 3'] = data['línea 3'].astype(float)
    #data['línea 3.1'] = data['línea 3.1'].astype(float)
    #data['Viabilidad final cultivo_cultivo_final'] = data['Viabilidad final cultivo_cultivo_final'].astype(float)
    for column in data.select_dtypes(include=[np.number]).columns:
        mean_value = data[column].mean()
        data[column].fillna(mean_value, inplace=True)
    for col in data.select_dtypes(include=['object']).columns:
        data[col] = clean_and_convert(data[col])
    return data


In [19]:
data = data_nan(df_dropped)
data_test = data_nan(df_test_dropped)
# Definir características (X) y objetivo (y)
X = data
X_test_final = data_test
y = target

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Lasso Regressor Pipeline
lasso_pipeline = Pipeline([
    ('imputer', SimpleImputer()),  # Estrategia se selecciona en GridSearch
    ('scaler', StandardScaler()),  # Escalador se selecciona en GridSearch
    ('model', Lasso())
])

# GridSearchCV para Lasso
lasso_param_grid = {
    'imputer__strategy': ['mean', 'median', 'most_frequent'],  # Varias estrategias de imputación
    'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler()],  # Varias opciones de escaladores
    'model__alpha': [0.1, 1, 10, 100, 1000],  # Términos de regularización
    'model__max_iter': [1000, 5000, 10000, 15000, 100000],     # Iteraciones máximas
    'model__fit_intercept': [True, False],  # Considerar intercepto
    'model__tol': [1e-5, 1e-4, 1e-3, 1e-2]  # Tolerancia para convergencia
}

lasso_grid_search = GridSearchCV(
    estimator=lasso_pipeline, 
    param_grid=lasso_param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1
)

lasso_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             n_jobs=-1,
             param_grid={'imputer__strategy': ['mean', 'median',
                                               'most_frequent'],
                         'model__alpha': [0.1, 1, 10, 100, 1000],
                         'model__fit_intercept': [True, False],
                         'model__max_iter': [1000, 5000, 10000, 15000, 100000],
                         'model__tol': [1e-05, 0.0001, 0.001, 0.01],
                         'scaler': [StandardScaler(), MinMaxScaler(),
                                    RobustScaler()]},
             scoring='neg_mean_squared_error')

In [22]:
best_model = lasso_grid_search.best_estimator_

In [24]:
best_model.fit(X, y)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler()),
                ('model', Lasso(alpha=10, tol=0.01))])

In [25]:
y_final_pred = best_model.predict(X_test_final)

In [28]:
data = pd.DataFrame({
    'LOTE': test_df['LOTE'].astype(int),  # Ensure LOTE is of integer type
    'PRODUCTO 1': y_final_pred  # Your predictions
})
data.sort_values(by='LOTE', ascending=True, inplace=True)

# Format the PRODUCTO 1 values to ensure correct decimal representation
data['PRODUCTO 1'] = data['PRODUCTO 1'].apply(lambda x: f"{x:.2f}".replace(',', '.'))

# Save the DataFrame to a text file with '|' as the delimiter
data.to_csv('../final/universityhack.predict', sep='|', header=False, index=False)